In [1]:
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pygments.lexers import j
import datetime

In [2]:
## test this - then add catigorical variables
class veeringPhase_frame:
    ## pandas data from to recieve phases from SQL RRP.
    ## frame is then processed to bin variables, define categorical variables
    def __init__(self,rawFrame):
        self.frame = rawFrame
        
    def Add_Date_column(self):
        self.frame['Date'] = self.frame['StartTime'].apply(lambda x: x.date())

    def CheckSigns(self,vars):
        for var in vars:
            self.frame[var] = abs(self.frame[var])

    def DefineBins(self,min_max_step):
        ## function takes a list [min, max, step] and defines the bin edges and labels
        min, max, step = min_max_step[0], min_max_step[1]+1, min_max_step[2]
        centres = list(np.arange(min,max,step))
        min = min - (0.5*step)
        max = max + (0.5*step)
        edges = list(np.arange(min,max,step))
        return centres, edges

        
    def Create_Bin_Variables(self,binVars):
        ## function takes a list of variables [varName, newVarName, [min,max,step] and creates a binned variable
        self.binVars = binVars
        for var in self.binVars:
            centres, edges = self.DefineBins(var[2])
            self.frame[var[1]] = pd.cut(self.frame[var[0]], bins=edges, labels=centres)
            
    def Create_Categorical_Variables(self,catVars, cat_threshold):
        ## function changes defined variables to a categorical type and removes categories where the count is < threshold
        self.catVars = catVars
        self.cat_threshold = cat_threshold
        for var in categoricalVariables:
            self.frame[var] = self.frame[var].astype('category')
            catCount = self.frame.groupby(var).size()
            catCount = catCount/sum(catCount)
            drop = catCount[catCount < self.cat_threshold].index
            self.frame[var] = self.frame[var].cat.remove_categories(drop)
        
        
class veeringDay_frame:
    def __init__(self,frame):
        self.frame = frame
        
    def Filter_Date(self,date_list):
        self.year = date_list[0]
        self.month = date_list[1]
        self.day = date_list[2]
        self.frame = self.frame.loc[self.frame.Date == datetime.date(year=year,month=month,day=day)]
        
        

In [3]:
config = {'username':'veering',
          'password':'veeringRaceReplay!',
          'host':'veeringq.ddns.net',
          'port':'3306',
          'database':'flyingjenny'
          }
cnx = mysql.connector.connect(**config)

phasesFrame = veeringPhase_frame(pd.read_sql_query('SELECT StartTime, SailingMode, RaceNum, TWS, TWA, BSP, HEEL, RUDDER, `BSPpol%`, `VMG%`, `BSP_trg%`, TWA_trg, AWA, FORESTAY, Chock, D1, D2, V1, Headstay, Mainsail, Headsail, Spinnaker FROM t_phase', cnx))

cnx.close()

/var/folders/b5/1kkh0fjj6cqg5n9rk3j623dr0000gn/T/ipykernel_7021/3984410005.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  phasesFrame = veeringPhase_frame(pd.read_sql_query('SELECT StartTime, SailingMode, RaceNum, TWS, TWA, BSP, HEEL, RUDDER, `BSPpol%`, `VMG%`, `BSP_trg%`, TWA_trg, AWA, FORESTAY, Chock, D1, D2, V1, Headstay, Mainsail, Headsail, Spinnaker FROM t_phase', cnx))


In [74]:
checkSignVars = ['TWA']

categoricalVars = ['SailingMode',
                        'RaceNum',
                        'Chock', 
                        'D1',
                        'D2',
                        'V1',
                        'Headstay',
                        'Mainsail',
                        'Headsail',
                        'Spinnaker']

binVars = [['TWS', 'TWS_bin',[1, 30, 1]],
           ['TWA', 'TWA_bin', [25, 165, 1]],
           ['HEEL', 'HEEL_bin', [0, 25, 1]],
           ['RUDDER', 'RUDDER_bin', [-1.6, 3.6, 0.2]],
           ['FORESTAY', 'FORESTAY_bin', [0.2, 2.2, 0.1]],
           ['BSP', 'BSP_bin', [3, 20, 0.1]],
           ['AWA', 'AWA_bin', [50, 120, 1]]]

date = [2023,6,9]


In [75]:
phasesFrame.Add_Date_column()
phasesFrame.CheckSigns(checkSignVars)
phasesFrame.Create_Categorical_Variables(categoricalVars,0.01)
phasesFrame.Create_Bin_Variables(binVars)
dayFrame = veeringDay_frame(phasesFrame.frame)
dayFrame.Filter_Date(date)




In [88]:
dayFrame.frame.TWS_bin.value_counts()

TWS_bin
16    41
17    40
15    32
14    26
19    13
12    11
18    11
13    10
20     8
21     4
11     4
10     1
22     1
23     0
24     0
25     0
26     0
27     0
28     0
29     0
1      0
2      0
9      0
8      0
7      0
6      0
5      0
4      0
3      0
30     0
Name: count, dtype: int64

In [82]:
dayFrame.frame.TWS_bin

0      14
1      14
2      11
3      11
4      16
       ..
197    17
198    17
199    17
200    17
201    17
Name: TWS_bin, Length: 202, dtype: category
Categories (30, int64): [1 < 2 < 3 < 4 ... 27 < 28 < 29 < 30]

In [64]:
phasesFrame.frame['Date'][0]

datetime.date(2023, 6, 9)

AttributeError: 'Series' object has no attribute 'type'